In [5]:
import pandas as pd
import numpy as np
import os
import sys
import scanpy as sc
import scipy
import time
import gget
from sklearn.decomposition import PCA
from importlib import reload

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import matplotlib.patheffects as pe

"""WARNING: disabling future warnings"""
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # and to omit tensorflow logging

""" Increase scanpy output"""
sc.settings.verbosity = 3

# local 
# import utils as ut
# reload(ut)

In [6]:
fullpath = "/nfs/turbo/umms-indikar/shared/projects/MC3R/processed_data/all_subclustered.h5ad"
raw = sc.read(fullpath)

pdf = raw.copy()
pdf

AnnData object with n_obs × n_vars = 33647 × 25060
    obs: 'batch', 'reference_embedding_C7_named', 'reference_embedding_C25_named', 'C25_named', 'C7_named', 'reference_embedding_C7_named_clean', 'reference_embedding_C25_named_clean', 'C25_named_clean', 'C7_named_clean', 'UMAP1', 'UMAP2', 'Diet', 'Sex', 'broad_type', 'n_genes', 'color', 'leiden', 'cluster_num', 'Cluster', 'cluster_cat', 'U1', 'U2', 'cell_type', 'neuron_clusters', 'neuron_cluster_num', 'neuron_cluster_name', 'neuron_cluster_cat', 'NU1', 'NU2'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Cluster', 'Cluster_colors', 'batch_colors', 'hvg', 'leiden', 'log1p', 'neighbors', 'neuron_clusters_colors', 'neuron_clusters_sizes', 'neuron_neighbors', 'paga', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances', 'neuron_neighbors_connectivities', 'neuron_neighbors_distances'

In [13]:
print(f"{pdf.var.shape=}")
pdf.var['is_MT'] = pdf.var.index.str.startswith("Mt")

pdf.var[['is_MT', 'highly_variable']].value_counts()

pdf.var.shape=(25060, 6)


is_MT  highly_variable
False  False              23992
       True                 997
True   False                 68
       True                   3
Name: count, dtype: int64

In [ ]:
break

In [22]:
res = []

for group_names, group in pdf.obs.groupby(['neuron_cluster_name', 'batch']):
    col_name = "_".join(list(group_names))
    expression = pdf[group.index, :].to_df()
    mean_expression = expression.mean(axis=0)
    mean_expression.name = col_name
    res.append(mean_expression)
    
res = pd.concat(res, axis=1)
res = res.reset_index()
outpath = "data/average_expression.csv"
res.to_csv(outpath, index=False)
res.head()

,index,C1_fasted_f,C1_fasted_m,C1_fed_f,C1_fed_m,C2_fasted_f,C2_fasted_m,C2_fed_f,C2_fed_m,C3_fasted_f,...,C6_fed_f,C6_fed_m,C7_fasted_f,C7_fasted_m,C7_fed_f,C7_fed_m,C8_fasted_f,C8_fasted_m,C8_fed_f,C8_fed_m
0,Xkr4,6.542037,6.707670,6.713317,6.723498,6.474062,6.648030,6.636553,6.697132,2.842699,...,6.782236,6.558888,7.415235,7.501945,6.994408,7.222719,6.872644,6.978508,6.772820,6.854861
1,Gm1992,2.904483,2.766563,2.704425,2.906262,2.706874,2.622447,2.727735,2.973604,0.426869,...,2.401357,2.220395,4.008621,3.386943,2.707013,3.239435,3.167731,3.058129,2.938307,2.853902
2,Gm19938,1.695348,1.850532,1.716756,1.736216,1.586273,1.732624,1.746167,1.686335,0.408319,...,1.456801,1.867134,2.569545,3.113293,1.528377,1.848124,1.716781,1.830696,1.333639,2.288886
3,Gm37381,0.000000,0.000000,0.000826,0.000984,0.001586,0.006904,0.000000,0.006697,0.000000,...,0.000000,0.000000,0.000000,0.022180,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Rp1,0.014791,0.013006,0.016847,0.020212,0.014815,0.027870,0.011429,0.022614,0.047005,...,0.000000,0.000000,0.000000,0.021933,0.043031,0.037713,0.000000,0.000000,0.135086,0.000000


In [23]:
res.shape

(25060, 33)